# Inference PointCloud MMDetection3D

In [1]:
import pickle
import os

import open3d as o3d
import numpy as np
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import mmdet3d

from mmdet3d.apis import inference_detector, init_model
# from sensus.utils.data_converter import pc2pc_object
from mmdet3d.utils import register_all_modules




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


# KITTI Dataset and MODEL

In [2]:
import numpy as np
from mmdet3d.structures.points import get_points_type
from mmdet3d.datasets.transforms.loading import LoadPointsFromFile

def pc2pc_object(pc, pipeline):
    pipeline_dict = {**pipeline[0]}
    assert pipeline_dict['type'] == 'LoadPointsFromFile'
    pipeline_dict.pop('type')
    pc_loader = LoadPointsFromFile(**pipeline_dict)

    pc = pc.reshape(-1, pc_loader.load_dim)
    pc = pc[:, pc_loader.use_dim]
    if pc_loader.norm_intensity:
            assert len(pc_loader.use_dim) >= 4, \
                f'When using intensity norm, expect used dimensions >= 4, got {len(pc_loader.use_dim)}'  # noqa: E501
            pc[:, 3] = np.tanh(pc[:, 3])
    attribute_dims = None

    if pc_loader.shift_height:
        floor_height = np.percentile(pc[:, 2], 0.99)
        height = pc[:, 2] - floor_height
        pc = np.concatenate(
            [pc[:, :3],
                np.expand_dims(height, 1), pc[:, 3:]], 1)
        attribute_dims = dict(height=3)

    if pc_loader.use_color:
        assert len(pc_loader.use_dim) >= 6
        if attribute_dims is None:
            attribute_dims = dict()
        attribute_dims.update(
            dict(color=[
                pc.shape[1] - 3,
                pc.shape[1] - 2,
                pc.shape[1] - 1,
            ]))

    points_class = get_points_type(pc_loader.coord_type)
    pc_object = points_class(
        pc, points_dim=pc.shape[-1], attribute_dims=attribute_dims)

    return pc_object, pc

In [27]:
register_all_modules()

## Build the model from a config file and a checkpoint file
config_kitti = '../../mmdetection3d/configs/second/second_hv_secfpn_8xb6-80e_kitti-3d-car.py'
checkpoint_kitti = '../../mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth'
config_dair = '../configs/dair_second/second_hv_secfpn_8xb6-80e_kitti-3d-car.py'
checkpoint_dair = '../../work_dirs/second_hv_secfpn_8xb6-80e_kitti-3d-car/epoch_40.pth'
pcd_path = 'DAIR/000000/000000.bin'

model_kitti = init_model(config_kitti, checkpoint_kitti, device='cuda:0')
result_kitti, data_kitti = inference_detector(model_kitti, pcd_path)

model_dair = init_model(config_dair, checkpoint_dair, device='cuda:0')
result_dair, data_dair = inference_detector(model_dair, pcd_path)


/home/javier/sensus-loci/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:92: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Loads checkpoint by local backend from path: ../../mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth
Loads checkpoint by local backend from path: ../../work_dirs/second_hv_secfpn_8xb6-80e_kitti-3d-car/epoch_40.pth


In [28]:
print(result_dair.pred_instances_3d.bboxes_3d.tensor)

tensor([[ 4.4778e+01, -3.0999e+01, -1.9028e+00,  1.8732e+00,  4.5216e+00,
          1.6140e+00, -4.4107e-02],
        [ 4.5072e+01,  1.2847e+01, -1.7582e+00,  2.0196e+00,  4.4543e+00,
          1.5488e+00,  1.5111e+00],
        [ 2.0279e+01, -3.8130e+00, -1.6659e+00,  1.8537e+00,  4.3446e+00,
          1.4824e+00, -1.5649e+00],
        [ 1.8140e+01,  2.2359e+00, -1.6748e+00,  2.0748e+00,  4.2751e+00,
          1.4786e+00,  4.6851e+00],
        [ 3.0104e+01,  1.7039e+01, -1.8616e+00,  1.9067e+00,  4.4397e+00,
          1.5570e+00,  1.5491e+00],
        [ 5.6480e+01,  2.5386e+01, -2.0134e+00,  2.0133e+00,  4.4245e+00,
          1.7138e+00,  1.6131e+00],
        [ 3.7032e+01, -7.9377e+00, -1.7243e+00,  1.8748e+00,  4.3192e+00,
          1.5334e+00,  4.6763e+00],
        [ 2.4175e+01, -7.7765e+00, -1.7014e+00,  1.8916e+00,  4.6678e+00,
          1.5334e+00,  4.7028e+00],
        [ 5.8449e+01,  1.6281e+01, -1.8318e+00,  1.9970e+00,  4.3651e+00,
          1.7495e+00,  1.5106e+00],
        [ 

## Lets draw

In [29]:
def generate_bbox(bbox_tensor):
    # Extract label info
    dimensions = bbox_tensor[3:6]
    position = bbox_tensor[0:3]
    rotation = bbox_tensor[6]

    print(dimensions)
    
    # Generate bbox
    box = o3d.geometry.TriangleMesh.create_box(width=dimensions[0], height=dimensions[1], depth=dimensions[2])

    # Width is x, height is -y and z is up
    box.paint_uniform_color([1.0, 0.0, 0.0]) # Set color to red

    # Translate box
    box.translate(position)

    # Translate box to center
    box.translate([-dimensions[0]/2, -dimensions[1]/2, 0])

    # Rotate box
    center = box.get_center()
    rotation = box.get_rotation_matrix_from_xyz((0, 0, -rotation)) # Notice the negative sign for rotation

    box.rotate(rotation, center=center)

    lines = o3d.geometry.LineSet.create_from_triangle_mesh(box)
    # Remove lines that connect non-adjacent points
    lines.lines = o3d.utility.Vector2iVector(np.array([[0, 1], [2, 0], [2, 3], [3, 1], [4, 5], [4, 6], [6, 7], [7, 5], [0, 4], [1, 5], [2, 6], [3, 7]]))
    lines.paint_uniform_color([1, 0, 0])

    return lines

In [30]:
# Read pc from bin file
with open(pcd_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)

PointCloud with 56285 points.


In [31]:
bboxes3d_kitti = result_kitti.pred_instances_3d.bboxes_3d.tensor.to('cpu').detach().numpy()

bbox_kitti = generate_bbox(bboxes3d_kitti[0])

# Draw bbox into point cloud
draw([pcd_bin, bbox_kitti], width=900, height=600, point_size=2)

[4.1206164 1.8063418 1.475353 ]
[Open3D INFO] Window window_8 created.


WebVisualizer(window_uid='window_8')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.6380479119032945
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.6380479119032945
[Open3D INFO] Sending init frames to window_8.


In [32]:
bboxes3d_dair = result_dair.pred_instances_3d.bboxes_3d.tensor.to('cpu').detach().numpy()

bbox_dair = generate_bbox(bboxes3d_dair[0])

# Draw bbox into point cloud
draw([pcd_bin, bbox_dair], width=900, height=600, point_size=2)

[1.8731527 4.5215507 1.6140195]
[Open3D INFO] Window window_9 created.


WebVisualizer(window_uid='window_9')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.8046029083453732
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.8046029083453732
[Open3D INFO] Sending init frames to window_9.


[3065:826][730221] (stun_port.cc:96): Binding request timed out from 163.117.150.x:46424 (enp4s0)
[3080:541][730221] (stun_port.cc:96): Binding request timed out from 163.117.150.x:39750 (enp4s0)
